In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain
import nltk
import sklearn
import scipy. stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd
import numpy as np
from itertools import groupby
import pickle

load data

In [ ]:
train=pd.read_table('训练语料/train/train_data.txt',names=['id','word','entity','tag'])
test=pd.read_table('训练语料/test/test_data.txt',names=['id','word','entity','tag'])
validf=pd.read_table('训练语料/test/test_data_label.txt', names=['id','word','entity','tag'])

In [ ]:
train=train.dropna(subset=['word'])

In [ ]:
len(train), len(test), len(validf)

In [ ]:
def remove_56_motor(df):
    l=df['tag']
    ldx56motor=[]
    for i in range(len(df)):
        if '56_Motor' in l[i] or 'TemporalConstraint' in l[i]:
            ldx56motor.append(i)
    newdf=df.drop(ldx56motor, axis=0)
    return newdf

In [ ]:
train=remove_56_motor(train)
validf=remove_56_motor(validf)
print(len(train),len(validf))

In [ ]:
labels=list(set(train['tag']))
labels.remove('O')
labels

In [ ]:
label_set=labels

In [ ]:
def dataset_id_list(pkl_file_type,num):
    pkl_file_name = 'nihss_'+ pkl_file_type +'_fold_'+str(num)
    with open('训练语料/train/' + pkl_file_name + '.pkl', 'rb') as f:
        nihss_train_fold_0 = pickle.load(f)
    id_list = []
    for i in nihss_train_fold_0:
        x = i['HADM_ID']
        id_list.append(x)
    return id_list

In [ ]:
# training set
traindf0 = train[train.id.isin(dataset_id_list('train',0))]
traindf1 = train[train.id.isin(dataset_id_list('train',1))]
traindf2 = train[train.id.isin(dataset_id_list('train',2))]
traindf3 = train[train.id.isin(dataset_id_list('train',3))]
traindf4 = train[train.id.isin(dataset_id_list('train',4))]

# validating set
testdf0 = train[train.id.isin(dataset_id_list('valid',0))]
testdf1 = train[train.id.isin(dataset_id_list('valid',1))]
testdf2 = train[train.id.isin(dataset_id_list('valid',2))]
testdf3 = train[train.id.isin(dataset_id_list('valid',3))]
testdf4 = train[train.id.isin(dataset_id_list('valid',4))]

In [ ]:
traindf0.head()

In [ ]:
traindf0.head()

df to list

In [ ]:
def df2list2(df):
    train_sents=[]
    for i in set(df['id']):
        ltrain=[]
        sdf=df[df['id']==i]
        for j in range(len(sdf)):
            s=tuple(sdf.iloc[j,1:4])
            #print(s)
            ltrain.append(s)
        train_sents.append(ltrain)
    return train_sents

In [ ]:
def word2features(sent,i):
    word=sent[i][0]
    postag=sent[i][1]
    
    features={
        'bias':1.0,
        'word.lower()':word.lower(),
        'word[-3:]':word[-3:],
        'word[-2:]':word[-2:],
        'word.isupper()':word.isupper(),
        'word.istitle()':word.istitle(),
        'word.isdigit()':word.isdigit(),
        #'postag[:2]':postag[:2],
    }
    if i>0:
        word1=sent[i-1][0]
        postag1=sent[i-1][1]
        features.update({
            '-1:word.lower()':word1.lower(),
            '-1:word.istitle()':word1.istitle(),
            '-1:word.isupper()':word1.isupper(),
            #'-1:postag':postag1,
            #'-1:postag[:2]':postag1[:2],
        })
    else:
        features['BOS']=True
        
    if i<len(sent)-1:
        word1=sent[i+1][0]
        #print(i,word1)
        postag1=sent[i+1][0]
        features.update({
            '+1:word.lower()':word1.lower(),
            '+1:word.istitle()':word1.istitle(),
            '+1:word.isupper()':word1.isupper(),
            #'+1:postag':postag1,
            #'+1:postag[:2]':postag1[:2],
        })
    else:
        features['EOS']=True
    
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
vali_set=df2list2(validf)
vali_set

extract features from data

training

In [ ]:
def training(X_train,y_train):
    %%time
    crf=sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    return crf

evaluation

In [ ]:
def sent2tag(sent_list):
    sent2tag_list=[]
    for i in sent_list:
        sent2tag_list=sent2tag_list+i
    return sent2tag_list

def tag2entity(sent_list):
    tag2entity_list=[]
    for j in sent2tag(sent_list):
        if j=='O':
            tag2entity_list.append([j])
        elif j[0]=='B':
            tag2entity_list.append([j])
        elif j[0]=='I':
            tag2entity_list[len(tag2entity_list)-1].append(j)
    return tag2entity_list

#def tag2entity_pred(sent_list,y_pred):
#    tag2entity_pred_list=[]
#    x=0
#    for i in tag2entity(sent_list):
#        tag2entity_pred_list.append(sent2tag(y_pred)[x:x+len(i)])
#        x+=len(i)
#    return tag2entity_pred_list
        
def entity2label(tag2entity_list):
    entity2label_list=[]
    for k in tag2entity_list:
        #print(k)
        l=[]
        for m in k:
            lm=m.split('-')
            l.append(lm[len(lm)-1])
        if len(set(l))==1:
            entity2label_list.append(l[0])
        elif len(set(l))>1:
            entity2label_list.append(','.join(set(l)))  
            #print(','.join(set(l)))
    return entity2label_list

In [ ]:
def tag2entity_pred(sent_list_test, sent_list_pred):
    tag2entity_pred_list=[]
    tag2entity_list=tag2entity(sent_list_test)
    sent2tag_pred_list=sent2tag(sent_list_pred)
    x=0
    for i in tag2entity_list:
        tag2entity_pred_list.append(sent2tag_pred_list[x:x+len(i)])
        x+=len(i)
    return tag2entity_pred_list

calculate precision recall

precision: corrected predicted nihss entitiy / all entity predicted as nihss

In [ ]:
def overall_perf(test_label,pred_label):
    tp=0
    for i in range(len(test_label)):
        if test_label[i]==pred_label[i] and test_label[i]!='O':
            tp+=1
    while 'O' in test_label:
        test_label.remove('O')
    rp=len(test_label)
    
    while 'O' in pred_label:
        pred_label.remove('O')
    pp=len(pred_label)
    precision=tp/pp
    recall=tp/rp
    f1=2*(precision*recall)/(precision+recall)
    overall_df=pd.DataFrame(data={'precision':precision, 'recall':recall, 'f1':f1}, index=['overall'])
    return overall_df

In [ ]:
def train_test_vali(traindf,testdf,validf):
    train_set=df2list2(traindf)
    test_set=df2list2(testdf)
    vali_set=df2list2(validf)
    
    X_train=[sent2features(s) for s in train_set]
    y_train=[sent2labels(s) for s in train_set]
    X_test=[sent2features(s) for s in test_set]
    y_test=[sent2labels(s) for s in test_set]
    X_vali=[sent2features(s) for s in vali_set]
    y_vali=[sent2labels(s) for s in vali_set]
    
    #crf=sklearn_crfsuite.CRF(
        #algorithm='lbfgs',
        #c1=0.1,
        #c2=0.1,
        #max_iterations=100,
        #all_possible_transitions=True
    #)
    #crf.fit(X_train, y_train)
    
    #labels=list(crf.classes_)
    #labels.remove('O')
    #label_set=[x.split('-')[1] for x in labels]
    label_set=list(set(traindf['tag']))
    label_set.remove('O')
    
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs', 
        max_iterations=100, 
        all_possible_transitions=True
    )
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
    }
    
    # use the same metric for evaluation
    f1_scorer = make_scorer(metrics.flat_f1_score, 
                            average='weighted', labels=label_set)

    # search
    rs = RandomizedSearchCV(crf, params_space, 
                            #cv=3,  #cv default value is 5
                            verbose=1, 
                            n_jobs=-1, 
                            n_iter=50, 
                            scoring=f1_scorer)
    rs.fit(X_train, y_train)
    crf=rs.best_estimator_
    y_pred_vali=crf.predict(X_vali)

    #y_pred=crf.predict(X_test)
    #f1_score=metrics.flat_f1_score(y_test, y_pred, average='weighted',labels=labels)
    #print('f1:',f1_score)
    
    y_vali_label=entity2label(tag2entity(y_vali))
    y_pred_vali_label=entity2label(tag2entity_pred(y_vali, y_pred_vali))
    
    performance=perfm(y_vali_label,y_pred_vali_label)
    overalldf=overall_perf(y_vali_label,y_pred_vali_label)
    performance=performance.append(overalldf)
    
    return performance

In [ ]:
def perfm(y_real,y_pred):
    TP_dict=dict()
    PP_dict=dict()
    RP_dict=dict()
    label_set = list(set(y_real))
    label_set.remove('O')
    
    for i in [TP_dict,PP_dict, RP_dict]:
        for j in label_set:
            i[j]=0
    
    for i in range(len(y_real)):
        if y_real[i]==y_pred[i] and y_real[i]!='O':
            TP_dict[y_real[i]]+=1
    for i in y_pred:
        if i in label_set:
            PP_dict[i]+=1
    #print(PP_dict)
    for i in y_real:
        if i in label_set:
            RP_dict[i]+=1
            
    precision_dict=dict()
    recall_dict=dict()
    f1_dict=dict()
    for i in label_set:
        if PP_dict[i]==0:
            precision_dict[i]=0
        else:   
            precision_dict[i]=TP_dict[i]/PP_dict[i]
        if RP_dict[i]==0:
            recall_dict[i]=0
        else:
            recall_dict[i]=TP_dict[i]/RP_dict[i]
        if (precision_dict[i]+recall_dict[i])==0:
            f1_dict[i]=0
        else:
            f1_dict[i]=2*(precision_dict[i]*recall_dict[i])/(precision_dict[i]+recall_dict[i])
    
    precision_df=pd.DataFrame(precision_dict, index=['precision'])
    recall_df=pd.DataFrame(recall_dict, index=['recall'])
    f1_df=pd.DataFrame(f1_dict, index=['f1'])
    perf_df=pd.concat([precision_df, recall_df,f1_df])
    perf_dft=pd.DataFrame(perf_df.values.T, index=perf_df.columns, columns=perf_df.index)
    return perf_dft

In [ ]:
train_test_vali(traindf1,testdf1,validf)

In [ ]:
y=0
for i,j,k in [(traindf0,testdf0,'fold0'),(traindf1,testdf1,'fold1'),(traindf2,testdf2,'fold2'),(traindf3,testdf3,'fold3'),(traindf4,testdf4,'fold4')]:
    print(y)
    if y==0:
        df=train_test_vali(i,j,validf)
        y+=1
    else:
        dfy=train_test_vali(i,j,validf)
        dfy.columns=[i+k for i in dfy.columns]
        df=pd.concat([df,dfy], axis=1, join_axes=[df.index])
        y+=1
df

In [ ]:
df['average_precision']=df.apply(lambda x: (x.precisionfold0+x.precisionfold1+x.precisionfold2+x.precisionfold3+x.precisionfold4)/5)
df['average_recall']=df.apply(lambda x: (x.recallfold0+x.recallfold1+x.recallfold2+x.recallfold3+x.recallfold4)/5)
df['average_f1']=df.apply(lambda x: (x.f1fold0+x.f1fold1+x.f1fold2+x.f1fold3+x.f1fold4)/5)

In [ ]:
df.to_csv('performance_vali_without56motor_tempreal_constraint test cv3 9 18.csv')